In [2]:
# import functions for model 
%run model_functions.ipynb
# %run jude_plot_code.py # not doing plotting anymore

In [3]:
# import libraries
from __future__ import print_function, division
import sys
import numpy as np
import random as random
import math as math
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from random import triangular
import scipy.stats as sst
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import jude_plot_code as plot

In [4]:
# set code preferences for spiny dogfish model
year_split = 2008 # in what year should the data be split into estimation (below year_split) or validation (equals to or after year_split)? currently leaving 10 years for forecasting/validation
abc_options = ['regression','rejection']
abc_pref = abc_options[1] # choose which approach to Approximate Bayesian Computation to use 
temperature_options = ['ROMS','in_situ'] # ROMS is still the incorrect run from the cod forecast challenge
temperature_pref = temperature_options[1]
print(abc_pref) # check it's correct, and remember Python starts from 0! 
print(temperature_pref)

rejection
in_situ


In [5]:
# import spiny dogfish data & look at columns 
dat_trawl = pd.read_csv("../processed-data/dogfish_prepped_data.csv")
list(dat_trawl.columns)

['haulid',
 'lengthclass',
 'spp',
 'numlengthclass',
 'region',
 'year',
 'common',
 'stratum',
 'stratumarea',
 'lat',
 'lon',
 'depth',
 'btemp']

In [6]:
# keep only needed columns and import temperature data if needed 
if temperature_pref=='ROMS':
    dat_trawl = dat_trawl[['haulid','numlengthclass', 'year', 'lat','lengthclass']] # keep only needed columns 
    dat_roms = pd.read_csv("~/github/SDM-convergence/data/haul_ROMS.csv", usecols = ['unique_id',  'temp_bottom', 'temp_surface']) # import ROMS data 
    dat_roms.rename({"unique_id":"haulid"},axis="columns",inplace=True) # fix column names 
    dat_trawl = pd.merge(dat_estimation, dat_roms, how="inner", on="haulid") # merge with trawl data. because this is an inner join, it will omit NOAA hauls with no ROMS data, and ROMS data with no matches in the species' survey dataframe
if temperature_pref=='in_situ':
    dat_trawl = dat_trawl[['haulid','numlengthclass', 'year', 'lat','lengthclass','btemp']]
    dat_trawl.rename({'btemp':'temp_bottom'},axis="columns",inplace=True)
list(dat_trawl.columns) 

['haulid', 'numlengthclass', 'year', 'lat', 'lengthclass', 'temp_bottom']

In [7]:
# filter trawl data 
dat_estimation = dat_trawl.loc[(dat_trawl['year'] <= year_split)] # use years before year_split for estimation

# check year filtering worked correctly
print(dat_trawl.year.min())
print(dat_trawl.year.max())

print(dat_estimation.year.min())
print(dat_estimation.year.max())

# round latitudes to integers
dat_estimation.lat = dat_estimation.lat.round().astype(np.int) # revisit and be more precise about rounding; currently rounding to nearest integer, so bands are defined as center points (35-degree band runs from 34.51 to 35.49)

1963
2018
1963
2008


/Users/afh/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
# taking care of missing data in both data frames
# AF: commenting this out because interpolation is a bit dodgy here. will just deal with NAs 
# dat_estimation=dat_estimation.interpolate(method ='linear', limit_direction ='forward')
# dat_estimation=dat_estimation.interpolate(method ='linear', limit_direction ='backward')
# dat_roms =dat_roms.interpolate(method ='linear', limit_direction ='forward')
# dat_roms=dat_roms.interpolate(method ='linear', limit_direction ='backward')


# USE DF = DF.DROPNA() INSTEAD

In [9]:
list(dat_estimation.columns) 

['haulid', 'numlengthclass', 'year', 'lat', 'lengthclass', 'temp_bottom']

In [10]:
# track number of latitudes--currently the spatial unit of analysis
yearsN = dat_trawl['year'].nunique()
latRange = np.arange(start=dat_estimation.lat.min(), stop=dat_estimation.lat.max()+1, step=1) # recall that the range family of functions automatically omit the "stop" value
latN = latRange.size # note that this is slightly different from Jude's code, which defined nn as lat.max-lat.min, which would be my latN-1
yearsTrainRange = np.arange(start=dat_estimation.year.min(), stop=dat_estimation.year.max()+1, step=1)
yearsTrainN = yearsTrainRange.size
yearsTestRange = np.arange(start=year_split+1, stop=dat_trawl.year.max()+1, step=1)
yearsTestN = yearsTestRange.size

print(latN)
print(latRange)
print(yearsN)
print(yearsTrainN)
print(yearsTrainRange)
print(yearsTestN)
print(yearsTestRange)

19
[29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
56
46
[1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976
 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990
 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008]
10
[2009 2010 2011 2012 2013 2014 2015 2016 2017 2018]


## List of changes to nomenclature in Jude's code

| Original name | New name | Meaning |
| --- | --- | --- |
| `nn` | `latN` | number of unique latitude values |
| `df` | `dat_estimation` | trawl survey dataset used for training |
| `df['NUMLEN']` | `dat_estimation['numlengthclass']` | column name |
| `n` | `Nobs` | number of observations in a patch and age class |
| `m` | `yearsTrainN` | number of years of data in training dataset |


In [11]:
# subdivide training data into a "dictionary" referenced by "patch" (latitude band) and life stage 
D={}
D1={}
# AF: what happens when all values are NA for mean() and sum() below? check that this isn't an issue

#extracting the data from the data frames  and storing according to the patch and stage. We start from the first to the last last patch (1:nn+1)  and when in each patch, we extract the number of species for  each life stage, the temperature for each patch (and compute the avaerage for the year)
for q in range(1,latN+1): # latN+1 because range() does not use the final value, i.e., range(1,3) equals [1,2]
    #Juveniles for patch 33+q( since the min patch is 34, we will start with patch 34 --to the maximum # AF: get rid of all fixed numerics here 
    D['J_patch'+ str(q)]=dat_estimation.loc[(dat_estimation['lat'] == (dat_estimation['lat'].min()-1)+ q) & (dat_estimation['lengthclass']=='smalljuv')]
    #total number of observations in each patch for each year
    Nobs=len(D['J_patch'+ str(q)].year.values)
    A=np.empty((Nobs, 3))
    Abun_TemJ=np.empty((yearsTrainN, 4)) # create empty object; Alexa adding a column for real year 
    A[:,0]=D['J_patch'+ str(q)].year.values 
    A[:,1]=D['J_patch'+ str(q)].temp_bottom.values
    A[:,2]=D['J_patch'+ str(q)].numlengthclass.values
    kJ=0 # indices 
    kY=0
    kA=0
    # get temperature and abundance data in each year 
    for i in range (0, yearsTrainN):
        Abun_TemJ[i,0]=kJ
        N=np.zeros(len(A[:,0]))
        T=np.zeros(len(A[:,0]))
        for j in range (0, len(A[:,0])):
            if A[j,0] == min(yearsTrainRange)+i:
               T[j]=A[j,1]
               N[j]=A[j,2]
            else:
               T[j]=float("nan")
               N[j]=float("nan")
     #   Abun_TemJ[i,1]=np.nan_to_num(T[T!=0].mean())# see notes in adult chunk below re: this method
        Abun_TemJ[i,1]=np.nanmean(T)
        Abun_TemJ[i,2]=np.nansum(N)
        Abun_TemJ[i,3]=yearsTrainRange[i] # preserve actual year value 
        kJ=kJ +1
        
#         Abun_TemJ[i,0]=kJ # add index value 
#         DD=D['J_patch'+ str(q)] 
#         TT=DD.loc[(DD['year'] == yearsTrainRange[i])] # further subdivide patch data by year 
#         temp1=DD.loc[(DD['year'] == yearsTrainRange[i])] # why is this different from the above line? ask Jude
#      #   print(temp1.temp_bottom.values) # AF: not sure why this is here 
#         Abun_TemJ[i,1]=temp1.temp_bottom.values.mean()
#         Abun_TemJ[i,2]=TT.numlengthclass.values.sum()
#         Abun_TemJ[i,3]=yearsTrainRange[i] # preserve actual year value 
        
#         kJ=kJ +1
    #After extracting teh temperature and calculating the mean value, we now save it
    D1['J_patch'+ str(q)]=Abun_TemJ
    
    
# now moving to Young Juveniles to perform teh same process as above
    D['Y_patch'+ str(q)]=dat_estimation.loc[(dat_estimation['lat'] == (dat_estimation['lat'].min()-1)+ q) & (dat_estimation['lengthclass']=='largejuv')]
    Nobs=len(D['Y_patch'+ str(q)].year.values)
    B=np.empty((Nobs, 3))
    Abun_TemY=np.empty((yearsTrainN, 4))
    B[:,0]=D['Y_patch'+ str(q)].year.values 
    B[:,1]=D['Y_patch'+ str(q)].temp_bottom.values
    B[:,2]=D['Y_patch'+ str(q)].numlengthclass.values
    for i in range (0, yearsTrainN):
        Abun_TemY[i,0]=kY
        N=np.zeros(len(B[:,0]))
        T=np.zeros(len(B[:,0]))
        for j in range (0, len(B[:,0])):
            if B[j,0] == min(yearsTrainRange)+i:
                T[j]=B[j,1]
                N[j]=B[j,2]
            else:
                T[j]=float("nan")
                N[j]=float("nan")
     #   Abun_TemY[i,1]=np.nan_to_num(T[T!=0].mean()) # see notes in adult chunk below re: this method
        Abun_TemY[i,1]=np.nanmean(T)
        Abun_TemY[i,2]=np.nansum(N)
        Abun_TemY[i,3]=yearsTrainRange[i] # preserve actual year value 

#         DD=D['Y_patch'+ str(q)]
#         TT=DD.loc[(DD['year'] == yearsTrainRange[i])]
#         temp1=DD.loc[(DD['year'] == yearsTrainRange[i])]
#         Abun_TemY[i,1]=temp1.temp_bottom.values.mean()
#         Abun_TemY[i,2]=TT.numlengthclass.values.sum()
#         Abun_TemY[i,3]=yearsTrainRange[i] # preserve actual year value 
        kY=kY +1
    D1['Y_patch'+ str(q)]=Abun_TemY
    
    
#Next we move to Adult and perform teh same as above
    D['A_patch'+ str(q)]=dat_estimation.loc[(dat_estimation['lat'] == (dat_estimation['lat'].min()-1)+ q) & (dat_estimation['lengthclass']=='adult')]
    Nobs=len(D['A_patch'+ str(q)].year.values)
    C=np.empty((Nobs, 3))
    Abun_TemA=np.empty((yearsTrainN, 4))
    C[:,0]=D['A_patch'+ str(q)].year.values 
    C[:,1]=D['A_patch'+ str(q)].temp_bottom.values
    C[:,2]=D['A_patch'+ str(q)].numlengthclass.values
    for i in range (0, yearsTrainN):
        Abun_TemA[i,0]=kA
        N=np.zeros(len(C[:,2]))
        T=np.zeros(len(C[:,1]))
        for j in range (0, len(C[:,0])): # copy over temperature and count data only from year i; iterates over all rows 
            if C[j,0] == min(yearsTrainRange)+i:
                T[j]=C[j,1]
                N[j]=C[j,2]
            else:
                T[j]=float("nan") # AF: imputing a zero here was causing erroneous temperature estimates below 
                N[j]=float("nan") # not sure if this is causing any issues yet
#        Abun_TemA[i,1]=np.nan_to_num(T[T!=0].mean()) # mean bottom temperature (***has lots of zeros bc of nan_to_num*** and creates very low values, I think there is a mistake here)
        Abun_TemA[i,1]=np.nanmean(T) # calculate mean bottemp for patch*stage*year; previously just setting to zero if missing 
        Abun_TemA[i,2]=np.nansum(N) # calculate sum individuals for patch*stage*year; previously just setting to zero if missing 
        Abun_TemA[i,3]=yearsTrainRange[i]
#         DD=D['A_patch'+ str(q)]
#         TT=DD.loc[(DD['year'] == yearsTrainRange[i])]
#         temp1=DD.loc[(DD['year'] == yearsTrainRange[i])]
#         Abun_TemA[i,1]=temp1.temp_bottom.values.mean()
#         Abun_TemA[i,2]=TT.numlengthclass.values.sum()
#         Abun_TemA[i,3]=yearsTrainRange[i] # preserve actual year value 
        kA=kA +1
    D1['A_patch'+ str(q)]=Abun_TemA
    
    # for future, write out D1 with correct column names so that it can be visually inspected, plotted, etc. 
    # this runs fine but I'm having trouble matching up the data in D1[patch] with the input df. right now the numbers are sane (orders of magnitude) but need to precisely compare D1 to dat_estimation. 

/Users/afh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: Mean of empty slice
/Users/afh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: Mean of empty slice
/Users/afh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:106: RuntimeWarning: Mean of empty slice


In [12]:
D1['J_patch15']

array([[0.00000000e+00, 6.71842105e+00, 0.00000000e+00, 1.96300000e+03],
       [1.00000000e+00, 5.46216216e+00, 0.00000000e+00, 1.96400000e+03],
       [2.00000000e+00, 5.24864865e+00, 0.00000000e+00, 1.96500000e+03],
       [3.00000000e+00, 5.18918919e+00, 0.00000000e+00, 1.96600000e+03],
       [4.00000000e+00, 5.51794872e+00, 0.00000000e+00, 1.96700000e+03],
       [5.00000000e+00, 6.22051282e+00, 0.00000000e+00, 1.96800000e+03],
       [6.00000000e+00, 6.22804878e+00, 2.00000000e+00, 1.96900000e+03],
       [7.00000000e+00, 6.30446429e+00, 0.00000000e+00, 1.97000000e+03],
       [8.00000000e+00, 6.63240741e+00, 0.00000000e+00, 1.97100000e+03],
       [9.00000000e+00, 7.27327586e+00, 0.00000000e+00, 1.97200000e+03],
       [1.00000000e+01, 7.64380952e+00, 0.00000000e+00, 1.97300000e+03],
       [1.10000000e+01, 8.62500000e+00, 0.00000000e+00, 1.97400000e+03],
       [1.20000000e+01, 6.94361702e+00, 0.00000000e+00, 1.97500000e+03],
       [1.30000000e+01, 7.92772277e+00, 0.00000000e

In [13]:
# THIS IS UNNECESSARY CODE TO GENERATE A LIST OF PATCH IDS, I'M JUST PROUD OF IT
# latcodes = np.arange(start=1, stop=latN+1, step=1)
# Alist=[]
# Jlist=[]
# Ylist=[]
# for z in range(0, latN):
#     tmpnameA = "A_patch"+str(latcodes[z])
#     tmpnameJ = "J_patch"+str(latcodes[z])
#     tmpnameY = "Y_patch"+str(latcodes[z])
#     Alist.append(tmpnameA)
#     Jlist.append(tmpnameJ)
#     Ylist.append(tmpnameY)

# patchcodes = Alist+Ylist+Jlist

In [14]:
patch_data=[]
# pull out D1 items into dataframes
for patch_id, a in D1.items():
    df = pd.DataFrame(a)
    df['patch_id']=patch_id
    patch_data.append(df)

# collapse into one df
D1_df = pd.concat(patch_data)

# fix column names--THIS WILL BREAK IF COLUMNS ARE CHANGED
D1_df.rename({0:'index',1:'temp_bottom',2:'abundance',3:'year'},axis="columns",inplace=True)

# write out to CSV
D1_df.to_csv("spiny_dogfish_training_data.csv") 
# thanks to shawn taylor @dataecologist for help converting dictionary to df

In [16]:
# now that data has been structured in a dictionary for the model, run the model
# we already imported the model functions at the top
#main script starts from here
# if __name__ == '__main__': # AF: ask Jude what this does -- apparently it denotes the main script--commenting out for now 
# Import the abundance data and data for the other variables e.g temperature
#  import groundfish_training AF: this isn't necessary anymore and I deleted "groundfish_training." from all the calls to D1 or D
# the total number of generations
T_FINAL = len(D1['J_patch1'][:,0])
#We simulate 20000 sets of parameters for for ABC, using non informatives priors (uniform priors
NUMBER_SIMS = 20000
#no of patches
no_patches=latN

# need to define this for simulation_population; eventually go into functions and replace them all with sensibly named objects (T_FINAL, no_patches)
rows=T_FINAL
cols=no_patches

# creating an array to store the number of juveniles, young juvenils and adults in each patch
N_J=np.ndarray(shape=(rows, cols), dtype=float, order='F') # rows are years and columns are patches 
N_Y=np.ndarray(shape=(rows, cols), dtype=float, order='F')
N_A=np.ndarray(shape=(rows, cols), dtype=float, order='F')
tempA = np.ndarray(shape=(rows, cols), dtype=float, order='F')
#storing data (secies abundance and temeprature time series data ) in the created arrays
for q in range(1,no_patches+1):
    i=q-1
    p=q
    N_J[:,i]=D1['J_patch'+ str(p)][:,2] # fill in the array with data from D1. column 2 in D1 holds abundance. column 0 contains indices (kJ, etc) and column 1 contains temperatures. 
    N_Y[:,i]=D1['Y_patch'+ str(p)][:,2]
    N_A[:,i]=D1['A_patch'+ str(p)][:,2]
    tempA[:,i]=D1['A_patch'+ str(p)][:,1] # only varies across patches, not life stages, so just need to save 1x 
#running ABC. See the function for details. returns all the observe summary statitics (OS)and simulated summary statistics (SS) in a matrix with first row corresponding to OS and the rest of the rows to SS as well as the parameter values that led to the simulated summary statistics.
param_save, Obs_Sim         = run_sim() # runs without error for spiny dogfish; Obs_Sim has lots of zeros but maybe that's correct? 

In [20]:
T_FINAL

46

In [18]:
print(param_save)
param_save.shape

[[2.01306985e+00 1.07694091e+02 1.21694345e+01 ... 5.86813503e-03
  9.70000919e-02 9.42250848e+02]
 [9.14574773e-01 1.61958423e+02 1.47321088e+01 ... 8.55064527e-02
  6.95592994e-02 7.00323089e+02]
 [8.91696669e-01 1.24934222e+02 7.72427885e+00 ... 9.74417437e-03
  7.26823997e-02 2.98528892e+02]
 ...
 [2.20653782e+00 1.22381028e+02 8.14634423e+00 ... 5.81203215e-03
  3.17492399e-02 4.34921623e+02]
 [1.75361468e+00 1.22232486e+02 1.07698444e+01 ... 8.72091572e-02
  3.87422027e-02 7.58093417e+02]
 [3.29553357e-01 1.70061564e+02 1.22240302e+01 ... 2.48492144e-03
  6.68821817e-02 7.31796637e+02]]


(20000, 10)

In [ ]:

#normalize the rows of Obs_sim to have NOS in row 1 and NSS in the remaining rows. Substract rows i=2:NUMBER_SIMS from row 1 of Obs_sim (whic contain OS).Compute the eucleadean distance (d) between NSS and NOS then use it along side tolerance (δ), to determine all parameters and NSS corresponding to d ≤ δ.Choose δ such that δ × 100% of the NUMBER_SIMS simulated parameters and NSS are selected. retain the parameters that made this threshold (library), the weights ot be used in local linear regression and the NSS that meets the threshold (stats)
library, dists, stats,stats_SS,  NSS_cutoff, library_index   = sum_stats(Obs_Sim, param_save) # CURRENTLY FAILING HERE BECAUSE OBS_SIM INCLUDES SOME NA VALUES, LEADING NORMALIZE() TO FAIL
# performing rejectio ABC. Note that if UMBER_SIMS is big enough, but rejection and regression ABC leads to teh same results.
if abc_pref=='rejection':
    result, HPDR=do_rejection(library)
    print('see the results below')
    print('Estimates from rejection is:', result)
    print('Estimated HPDR from rejection is :', HPDR)
# Next we have regression ABC, perform it if only you are not performing rejection ABC above. Gives better results for NUMBER_SIMS small. I have commented it.
#  if abc_pref=='regression':
#      library_reg=do_logit_transformation(library, param_bound)LJ=34, Ly=68, Linf=200 # AFH: still commented out because the code is a little messed up
#      result_reg, HPDR_reg=do_kernel_ridge(stats, library_reg, param_bound)
PARAMS1={}
print(result[2]) # what are each of these? can we add a name column? 
PARAMS1 = {"L_0":result[0] , "L_inf": result[1],"L_J": 39.75,"L_Y": 67.5, "Topt": result[2], "width": result[3], "kopt": result[4],"xi":result[5], "m_J": result[6], "m_Y":result[7] , "m_A": result[8], "K": result[9]}

N_J1, N_Y1, N_A1 = simulation_population(PARAMS1) #AF: it appears that N_J1 contains the simulations for juveniles--all of them, not just patch 1? 


In [ ]:
PARAMS1

In [ ]:
# save parameter data 
import csv
with open('parameter_estimates.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in PARAMS1.items():
       writer.writerow([key, value])
# sheer wizardry 

In [ ]:
# trying to write out results of model to plot in R 
#print(type(N_J1))
#testdf = pd.DataFrame(N_J1)

#print(nn)
#print(testdf) # I think this has patch as columns and years as rows. would be nice to preserve the real year and patch IDs 

# Jude's code to write out to df 

print(PARAMS1) # write this out too! 

yr=[]
lat=[]
stage=[]
abun=[]
for p in range(0, latN): 
    for q in range(0, yearsTestN):
        abun.append(N_J1[:,p][q]) # N_J1 I think has a row for each year, both training and testing. Is this getting the correct year?? 
        lat.append(latRange.min()+p) #lat.append(36+p)
        yr.append(year_split+q+1) #yr.append(2013+q) # adding 1 because I'm afraid to start range() at 1 in case it messes up the indexing
        stage.append('smalljuv')
        abun.append(N_Y1[:,p][q])
        lat.append(latRange.min()+p) #lat.append(36+p)
        yr.append(year_split+q+1) #yr.append(2013+q)
        stage.append('largejuv')
        abun.append(N_A1[:,p][q])
        lat.append(latRange.min()+p) #lat.append(36+p)
        yr.append(year_split+q+1) #yr.append(2013+q)
        stage.append('adult')
df=pd.DataFrame({'Year': yr, 'Latitude':lat, 'Stage': stage, 'Abundance': abun})
print(df)

df.to_csv('spiny_dogfish_out.csv')

In [ ]:
# #Importing a file call plot to plot the results.
# # print('i just imported a plot')
# for q in range(1,no_patches+1):
#     i=q-1
#     p=q
#     plot.do_realdata(N_J1[:,i], N_J[:,i],  'J_abun_rej'+ str(p))
#     #plot.do_scatter(N_J1[:,i], N_J[:,i],  'J_abun_scatter'+ str(p))
#     plot.do_realdata(N_Y1[:,i], N_Y[:,i],  'Y_abun_rej'+ str(p))
#     #plot.do_scatter(N_Y1[:,i], N_Y[:,i],  'Y_abun_scatter'+ str(p))
#     plot.do_realdata(N_A1[:,i], N_A[:,i],  'A_abun_rej'+ str(p))
# #plot.do_scatter(N_A1[:,i], N_A[:,i],  'A_abun_scatter'+ str(p))
# ################################################################
# # plot the figures below if you willl like to plot the heatmap
#     NJ1=N_J1.transpose()
#     NJ=N_J.transpose()
#     NY1=N_Y1.transpose()
#     NY=N_Y.transpose()
#     NA1=N_A1.transpose()
#     NA=N_A.transpose()
#     print(NJ1.shape)
#     ax=sns.heatmap(NJ1, cmap="Greys", xticklabels=True, yticklabels=True,  cbar_kws={'label': 'Abundance'})
#     plt.xlabel("Year")
#     plt.ylabel("Latitude")
#     ax.set_xticklabels(pd.Series(range(1980, 2012)))
#     ax.set_yticklabels(pd.Series(range(34, 46)))
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#     ax.figure.savefig("sim_J.png", bbox_inches='tight')
#     plt.close()
# #############################################################
#     ax = sns.heatmap(NJ, cmap="Greys",  xticklabels=True, yticklabels=True, cbar_kws={'label': 'Abundance'})
#     plt.xlabel("Year")
#     plt.ylabel("Latitude")
#     ax.set_xticklabels(pd.Series(range(1980, 2012)))
#     ax.set_yticklabels(pd.Series(range(34, 46)))
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#     ax.figure.savefig("Obs_J.png", bbox_inches='tight')
#     plt.close()
# ##########################################################
#     ax = sns.heatmap(NY1, cmap="Greys", xticklabels=True, yticklabels=True,  cbar_kws={'label': 'Abundance'})
#     plt.xlabel("Year")
#     plt.ylabel("Latitude")
#     ax.set_xticklabels(pd.Series(range(1980, 2013)))
#     ax.set_yticklabels(pd.Series(range(34, 46)))
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#     ax.figure.savefig("Sim_Y.png", bbox_inches='tight')
#     plt.close()
# ##########################################################
#     ax = sns.heatmap(NY, cmap="Greys", xticklabels=True, yticklabels=True,  cbar_kws={'label': 'Abundance'})
#     plt.xlabel("Year")
#     plt.ylabel("Latitude")
#     ax.set_xticklabels(pd.Series(range(1980, 2013)))
#     ax.set_yticklabels(pd.Series(range(34, 46)))
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#     ax.figure.savefig("Obs_Y.png", bbox_inches='tight')
#     plt.close()
# ############################################################
#     ax = sns.heatmap(NA1, cmap="Greys", xticklabels=True, yticklabels=True, cbar_kws={'label': 'Abundance'})
#     plt.xlabel("Year")
#     plt.ylabel("Latitude")
#     ax.set_xticklabels(pd.Series(range(1980, 2013)))
#     ax.set_yticklabels(pd.Series(range(34, 46)))
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#     ax.figure.savefig("Sim_A.png", bbox_inches='tight')
#     plt.close()
# ############################################################
#     ax = sns.heatmap(NA, cmap="Greys",  xticklabels=True, yticklabels=True, cbar_kws={'label': 'Abundance'})
#     plt.xlabel("Year")
#     plt.ylabel("Latitude")
#     ax.set_xticklabels(pd.Series(range(1980, 2013)))
#     ax.set_yticklabels(pd.Series(range(34, 46)))
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#     ax.figure.savefig("Obs_A.png", bbox_inches='tight')
#     plt.close()

